# LSTM

A LSTM-, long short term memory, cell is a more sofisticated type of RNN. RNNs are normally used to treat serialized data, and LSTMs are better at treating long term dependencies than traditional vanilla LSTMs.

**Note** As I already showed how to calculate fully connected neural networks by hand, I will not do so here. Instead all numerical examples are coded with python and numpy.

**Given:**
The LSTM equations:

$$c_v = h_{t-1} + x_t$$
$$f_t = \sigma (W_f c_v + b_f)$$
$$i_t = \sigma (W_i c_v + b_i)$$
$$\hat{c_t} = tanh (W_c c_v + b_c)$$
$$o_t = \sigma (W_o c_v + b_o)$$



$$c_t = f_t \odot c_{t-1} + i_t \odot \hat{c_t}$$
$$h_t = o_t \odot tanh(c_t)$$


and the input text string "Neural Networks are fun"

**Find:**
* The forward pass that passes every component of the input through an LSTM cell. (Multiple cell pass through)

* The backward pass that propagates the loss through a single cell. (Single example pass through)

## Forward pass
Here I implement and show a simple forward pass in an LSTM. The forward pass through multiple timesteps is done by feeding the lstm with the c, and h output from the previous iteration along with the x-vector of the current iteration. This is all done in a for-loop.

In [2]:
import numpy as np

#Onehot encode data
string = "Neural networks are fun"
vocab = list(set(string.split()))
word_to_idx = {w:i for i,w in enumerate(vocab)}
idx_to_word = {i:w for i,w in enumerate(vocab)}
x = []
for word in string.split():
    x_i = np.zeros((len(vocab),1))
    x_i[word_to_idx[word]] = 1
    x.append(x_i)

    
cell_size = len(vocab)

W_f = np.random.normal(0,1,(cell_size, cell_size))
b_f = np.random.normal(0,1,(cell_size,1))

W_i = np.random.normal(0,1,(cell_size, cell_size))
b_i = np.random.normal(0,1,(cell_size,1))

W_g = np.random.normal(0,1,(cell_size, cell_size))
b_g = np.random.normal(0,1,(cell_size,1))

W_o = np.random.normal(0,1,(cell_size, cell_size))
b_o = np.random.normal(0,1,(cell_size,1))


def sigmoid(x, w, b):
    z = w@x + b
    return np.exp(z)/(np.exp(z) + 1)

def tanh(x,w,b):
    z = w@x + b
    return np.tanh(z)

def lstm(x, c_t, h_t):
    cv = h_t + x_i
    f_t = sigmoid(cv, W_f, b_f)
    i_t = sigmoid(cv,W_i, b_i)
    g_t = tanh(cv, W_g, b_g)
    o_t = sigmoid(cv, W_o, b_o)
    c_t = c_t*f_t + i_t*g_t
    h_t = o_t*np.tanh(c_t)
    return c_t, h_t

def softmax(x):
    sf = np.exp(x)
    sf /= sf.sum(keepdims=True)
    return sf
def cross_entropy_loss(x,targets):
    ce = targets * np.log(x)
    ce = ce.sum()
    return -ce
#Initializing cell state and hidden state to zero
c_t = np.zeros((cell_size,1))
h_t = np.zeros((cell_size,1))
#Doing the actual forward pass
for x_i in x[:-1]:
    print(f"Passing word '{idx_to_word[np.argmax(x_i)]}' through LSTM")
    c_t, h_t = lstm(x, c_t, h_t)
    
y_gt = x[-1]
y_pred = softmax(h_t)
print(f"\n\nPredicted vector: {y_pred}")
loss = cross_entropy_loss(y_pred, y_gt)
print(f"Loss: {loss}")


Passing word 'Neural' through LSTM
Passing word 'networks' through LSTM
Passing word 'are' through LSTM


Predicted vector: [[0.29021869]
 [0.17640258]
 [0.28724179]
 [0.24613694]]
Loss: 1.237120547437759


## Backward pass
To utilize the chain rule for backward pass I must calculate the derivatives within the LSTM. I will not write up the derivative of sigmoid/tanh here as I believe that belongs to a different topic. With the derivatives provided here, the derivative with respect to the cost function can be found using the chain rule.

$$\frac{\partial h_t}{\partial c_t} = o_t \odot (1-tan(c_t)^2)$$

$$\frac{\partial h_t}{\partial o_t} = tanh(c_t)$$

$$\frac{\partial c_t}{\partial c_{t-1}} = f_t$$
$$\frac{\partial c_t}{\partial f_t} = c_{t-1}$$
$$\frac{\partial c_t}{\partial i_t} = g_t$$
$$\frac{\partial c_t}{\partial g_t} = i_t$$

All the gates i, f, g, o will have the same general structure for derivative. I will define the general derivative of these with $y_t$ as output.

$$\frac{\partial y_t}{\partial W} = \frac{\partial y_t}{\partial z_t} x_t^T$$
$$\frac{\partial y_t}{\partial b} = \frac{\partial y_t}{\partial z_t}$$


To illustrate the backpropagation I will pass through only one timestep, but to do so through multiple time steps, one must simply cache the internal activations along the way and let 
$$\frac{\partial C}{\partial c_{t-1}} = \frac{\partial C}{\partial h_{t-1}}\frac{\partial h_{t-1}}{\partial c_{t-1}} + \frac{\partial C}{\partial c_t}\frac{\partial c_t}{\partial c_{t-1}}$$.

Thus backwards pass through more than one cell should not bee too difficult.



I implemented the above equations in numpy and used the finite difference gradient approximation test from assignment 2 to verify my calculated gradients.

### Model definition

In [ ]:
def sigmoid_backward(a):
    return a*(1-a)

#Defining LSTM with forward and backward pass
class LSTM():
    def __init__(self,cell_size):
        #Initializing weights
        self.W_f = np.random.normal(0,1,(cell_size, cell_size))
        self.b_f = np.random.normal(0,1,(cell_size,1))

        self.W_i = np.random.normal(0,1,(cell_size, cell_size))
        self.b_i = np.random.normal(0,1,(cell_size,1))

        self.W_g = np.random.normal(0,1,(cell_size, cell_size))
        self.b_g = np.random.normal(0,1,(cell_size,1))

        self.W_o = np.random.normal(0,1,(cell_size, cell_size))
        self.b_o = np.random.normal(0,1,(cell_size,1))
    def forward(self,x, c_t, h_t):
        self.c_t0 = c_t
        self.cv = h_t + x_i
        self.f_t = sigmoid(self.cv, self.W_f, self.b_f)
        self.i_t = sigmoid(self.cv,self.W_i, self.b_i)
        self.g_t = tanh(self.cv, self.W_g, self.b_g)
        self.o_t = sigmoid(self.cv, self.W_o, self.b_o)
        
        self.c_t = c_t*self.f_t + self.i_t*self.g_t
        self.h_t = self.o_t*np.tanh(self.c_t)
        return self.c_t, self.h_t
    def backward(self, y_pred, y_gt):
        delta = y_pred - y_gt

        d_ct = delta*self.o_t*(1-np.tanh(self.c_t)**2) #Tanh backward
        d_ot = delta*np.tanh(self.c_t)

        d_ft = d_ct*self.c_t0
        d_it = d_ct*self.g_t
        d_gt = d_ct*self.i_t

        self.d_wi = d_it*sigmoid_backward(self.i_t)@self.cv.transpose()
        self.d_wf = d_ft*sigmoid_backward(self.f_t)@self.cv.transpose()
        self.d_wg = d_gt*(1-self.g_t**2)@self.cv.transpose() #tanh backward
        self.d_wo = d_ot*sigmoid_backward(self.o_t)@self.cv.transpose()

        self.d_bi = d_it*sigmoid_backward(self.i_t)
        self.d_bf = d_ft*sigmoid_backward(self.f_t)
        self.d_bg = d_gt*(1-self.g_t**2)
        self.d_bo = d_ot*sigmoid_backward(self.o_t)
        
    def weights(self):
        return [self.W_f, self.W_o, self.W_g, self.W_i]
    def weight_grads(self):
        return [self.d_wf, self.d_wo, self.d_wg, self.d_wi]

### Model test

In [3]:
#Data vector
x = np.zeros((cell_size,1))
x[0] = 1

lstm = LSTM(cell_size)

#Initializing cell state and hidden state to zero
c_t0 = np.zeros((cell_size,1))
h_t0 = np.zeros((cell_size,1))

c_t1, h_t1 = lstm.forward(x, c_t0, h_t0)

y_pred = softmax(h_t1)

lstm.backward(y_pred, y_gt)


epsilon = 1e-4
for layer_idx,(w, dw) in enumerate(zip(lstm.weights(),lstm.weight_grads())):
    
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            
            orig = w[i, j].copy()
            w[i, j] = orig + epsilon
            c_t1, h_t1 = lstm.forward(x, c_t0, h_t0)
            y_pred = softmax(h_t1)
            cost1 = cross_entropy_loss(y_pred, y_gt)
            
            w[i, j] = orig - epsilon
            c_t1, h_t1 = lstm.forward(x, c_t0, h_t0)
            y_pred = softmax(h_t1)
            cost2 = cross_entropy_loss(y_pred, y_gt)
            gradient_approximation = (cost1 - cost2) / (2 * epsilon)
            
            w[i, j] = orig
            
            # Actual gradient
            c_t1, h_t1 = lstm.forward(x, c_t0, h_t0)
            lstm.backward(softmax(h_t1), y_pred)
            difference = gradient_approximation - dw[i,j]
            assert abs(difference) <= epsilon**2,\
                f"Calculated gradient is incorrect. " \
                f"Layer IDX = {layer_idx}, i={i}, j={j}.\n" \
                f"Approximation: {gradient_approximation}, actual gradient: {dw[i, j]}\n" \
                f"If this test fails there could be errors in your loss function, " \
                f"forward function or backward function"
    print(f"Successfull for weights {layer_idx}")

Successfull for weights 0
Successfull for weights 1
Successfull for weights 2
Successfull for weights 3
